In [85]:
from parse_hh_data import download, parse
from bs4 import BeautifulSoup
import re

def preprocess_vacancy_data(url):
    pattern = r'.*\/(\d+)\?.*'
    vacancy_id = re.sub(pattern, r'\1', url)
    vacancy = download.vacancy(vacancy_id)
    vacancy['key_skills'] = [i['name'] for i in vacancy['key_skills']]
    keys_to_keep = ['name', 'description', 'key_skills', 'salary', 'experience']

    # Create a new dictionary with only the specified keys
    vacancy = {key: vacancy[key] for key in keys_to_keep if key in vacancy}

    # Parse HTML
    soup = BeautifulSoup(vacancy['description'], 'html.parser')

    # Extract text
    text_split = soup.get_text()

    text_split = re.split(r'\s{2,}|;|[.!?]', text_split)

    
    parsed_description = [i.lower().strip() for i in text_split if i != '']

    vacancy['description'] = '\n'.join(parsed_description)
    vacancy['sentences'] = parsed_description
    vacancy['name'] = vacancy['name'].lower()
    vacancy['key_skills'] = [i.lower() for i in vacancy['key_skills']]

    return vacancy

url = "https://hh.ru/vacancy/96434709?query=DevOps-%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80&hhtmFrom=vacancy_search_list"
preprocessed_vacancy = preprocess_vacancy_data(url)
print(preprocessed_vacancy)


{'name': 'devops engineer', 'description': 'привет) мы компания пик - крупнейший застройщик в россии\nмы занимаемся автоматизацией всех функций компании и делаем строительную отрасль технологичной: разрабатываем системы, хранилища данных, мобильные приложения, сайты, боты\nвсё это для того, чтобы создавать лучшие дома\nсейчас мы ищем devops инженера в команду аналитики и отчётности\nдля наших сотрудников:\nработа в крупной стабильной компании с инновационными проектами\nофициальное трудоустройство по тк рф\nконкурентная заработная плата\nудаленный формат работы возможность для быстрого роста и развития вместе с командой\nсобственный портал обучения и электронная библиотека\nкоманда профессионалов, которая всегда придет на помощь\nчем предстоит заниматься:\nадминистрирование и оптимизация инфраструктуры серверов, вкл\nbi-системы (например, fine bi) на linux и windows\nуправление базами данных для обеспечения их производительности и доступности\nразработка и поддержка систем мониторинга 

In [86]:
from joblib import load

import pandas as pd
from fuzzywuzzy import fuzz

binar_model = load('/home/andrew/works/VacancySkillsEstimator/notebooks/clf.joblib')
tfidf_vectorizer = load('/home/andrew/works/VacancySkillsEstimator/notebooks/tfidf_vectorizer.joblib')


def predict_sentences(sentences):
    # Make predictions for each sentence
    sentences = tfidf_vectorizer.transform(sentences)
    predictions = binar_model.predict(sentences)

    return predictions

sentences = preprocessed_vacancy['sentences']


# Get predictions for the sentences
predictions = predict_sentences(sentences)
replacement_dict = {0: False, 1: True}

# Замена значений в массиве с помощью словаря
arr_replaced = np.array([replacement_dict[val] for val in predictions])

preprocessed_vacancy['requirements_predict'] = np.array(sentences)[arr_replaced]



# Создание пустого DataFrame для матрицы
matrix_df = pd.DataFrame(index=preprocessed_vacancy['requirements_predict'], columns=df_gb.index)

# Установка параметров для окна
threshold = 50  # Порог сходства

# Проходимся по каждому требованию и каждому элементу столбца 'other' и заполняем матрицу
for req in preprocessed_vacancy['requirements_predict']:
    for idx, other_text in df_gb['other'].items():
        max_similarity = 0
        window_size = len(req)
        for i in range(len(other_text) - window_size + 1):
            window = other_text[i:i+window_size].lower()
            similarity = fuzz.partial_ratio(req.lower(), window)
            max_similarity = max(max_similarity, similarity)
        if max_similarity >= threshold:
            matrix_df.at[req, idx] = True
        else:
            matrix_df.at[req, idx] = False

# Заполнение пропущенных значений False
matrix_df = matrix_df.fillna(False)


In [62]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)

def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords and punctuation
    tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
    # Join tokens back into text
    preprocessed_text = ' '.join(tokens).lower()
    return preprocessed_text

def preprocess_csv_gb(csv_file_path):
    # Download NLTK resources


    # Load DataFrame
    df_gb = pd.read_csv(csv_file_path)

    # Rename column for convenience
    df_gb['stack'] = df_gb['Технологии, инструменты']

    # Select only required columns
    df_gb = df_gb[['index', 'name', 'url', 'stack', 'summary', 'other']]

    # Preprocess 'stack' column
    df_gb['stack'] = df_gb['stack'].str.lower().str.split(',')

    # Define stopwords and punctuation


    # Preprocess 'summary' column
    df_gb['summary'] = df_gb['summary'].apply(preprocess_text)

    # Preprocess 'other' column
    df_gb['other'] = df_gb['other'].apply(preprocess_text)

    df_gb['name'] = df_gb['name'].apply(lambda x: x.lower())

    return df_gb

# Пример использования модуля
csv_file_path = '/home/andrew/works/VacancySkillsEstimator/data/resultv1.csv'
preprocessed_df_gb = preprocess_csv_gb(csv_file_path)


[nltk_data] Downloading package punkt to /home/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
import pandas as pd

def preprocess_skills(skills):
    return ';'.join(skill.strip().lower() for skill in skills[1:-1].split(','))

def preprocess_it_csv(csv_file_path):
    # Load DataFrame
    df_it = pd.read_csv(csv_file_path, sep=';')

    # Remove rows with NaN values in 'description' and 'name' columns
    df_it = df_it[~df_it['description'].isna()].reset_index(drop=True)
    df_it = df_it[~df_it['name'].isna()].reset_index(drop=True)

    # Select only required columns
    df_it = df_it[['name', 'description', 'hard_skills']]

    # Preprocess 'hard_skills' column
    df_it['hard_skills'] = df_it['hard_skills'].apply(preprocess_skills)

    # Preprocess 'description' column
    df_it['description'] = df_it['description'].apply(preprocess_text)

    # Lowercase 'name' column
    df_it['name'] = df_it['name'].apply(lambda x: x.lower())

    return df_it

# Пример использования модуля
csv_file_path = '/home/andrew/works/VacancySkillsEstimator/data/it.csv'
preprocessed_df_it = preprocess_it_csv(csv_file_path)


In [64]:
from fuzzywuzzy import fuzz
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler


def levenshtein_distance_sort(row, it_name):
    return fuzz.ratio(it_name, row['name'])

def calculate_skill_similarity(row, it_skills):
    gb_skills = row['stack']
    
    # Объединяем навыки в одну строку
    gb_skills_str = ' '.join(gb_skills)
    it_skills_str = ' '.join(it_skills)
    
    # Расстояние Левенштейна между строками навыков
    levenshtein_distance = fuzz.ratio(gb_skills_str, it_skills_str)
    
    # Максимальное пересечение множеств между навыками
    gb_skills_counter = Counter(gb_skills)
    it_skills_counter = Counter(it_skills)
    intersection = gb_skills_counter & it_skills_counter
    max_intersection = sum(intersection.values())
    
    return levenshtein_distance, max_intersection

def calculate_tfidf_similarity(df_gb, df_it, vacancy):
    # Объединяем все текстовые данные из df_gb в один список
    vacancy_texts = [vacancy['name'] + vacancy['description'] + ' '.join(vacancy['key_skills'])]

    gb_texts = list(df_it['description'] + [' '.join(i) for i in df_it['hard_skills']]) + vacancy_texts + list(df_gb['summary'] + df_gb['other']+ [' '.join(i) for i in df_gb['stack']])

    # Создаем TF-IDF векторизатор
    tfidf_vectorizer_gb = TfidfVectorizer()

    # Применяем TF-IDF векторизацию к текстовым данным из df_gb
    tfidf_matrix = tfidf_vectorizer_gb.fit_transform(gb_texts)

    # Преобразуем описание из df_it в TF-IDF вектор
    tfidf_matrix_it = tfidf_matrix[:len(df_it)+1]
    tfidf_matrix_gb = tfidf_matrix[len(df_it)+1:]

    # Вычисляем косинусное сходство между описанием из df_it и каждым элементом из df_gb
    cosine_similarities = cosine_similarity(tfidf_matrix_it[-1], tfidf_matrix_gb)

    # Добавляем рейтинг сходства в df_gb
    df_gb['tfidf_description'] = cosine_similarities[0]

    return df_gb

def preprocess_vacancy_data(vacancy):
    vacancy['description'] = preprocess_text(vacancy['description'])
    vacancy['name'] = vacancy['name'].lower()
    return vacancy

def preprocess_and_match_vacancy(preprocessed_vacancy, df_gb, df_it):
    # Preprocess vacancy data
    preprocessed_vacancy = preprocess_vacancy_data(preprocessed_vacancy)

    # Calculate Levenshtein distance between vacancy name and company names in df_gb
    df_gb['levenshtein_distance'] = df_gb.apply(levenshtein_distance_sort, args=(preprocessed_vacancy['name'],), axis=1)

    # Extract IT skills from vacancy
    it_skills = preprocessed_vacancy['key_skills']

    # Calculate skill similarity between vacancy and job descriptions in df_gb
    df_gb['levenshtein_distance_stack'], df_gb['max_intersection_stack'] = zip(*df_gb.apply(calculate_skill_similarity, args=(it_skills,), axis=1))

    # Calculate TF-IDF similarity between vacancy description and job descriptions in df_gb
    df_gb = calculate_tfidf_similarity(df_gb, df_it, preprocessed_vacancy)

    # Нормализация метрик
    scaler = MinMaxScaler()
    metrics_to_normalize = ['levenshtein_distance', 'levenshtein_distance_stack', 'max_intersection_stack', 'tfidf_description']
    df_gb['total_score'] = scaler.fit_transform(df_gb[metrics_to_normalize]).sum(axis=1)
    df_gb = df_gb.sort_values(by='total_score', ascending=False).reset_index(drop=True)

    return df_gb

df_gb = preprocess_and_match_vacancy(preprocessed_vacancy, preprocessed_df_gb, preprocessed_df_it)


In [87]:
preprocessed_vacancy['requirements_predict']

array(['администрирование и оптимизация инфраструктуры серверов, вкл',
       'bi-системы (например, fine bi) на linux и windows',
       'управление базами данных для обеспечения их производительности и доступности',
       'разработка и поддержка систем мониторинга и логирования событий для всей инфраструктуры (сервера, базы данных, приложения)',
       'обеспечение безопасности систем, включая резервное копирование данных',
       'помощь разработчикам в настройке тестовых и разработческих сред, включая поддержку контейнеров и виртуализацию',
       'диагностика и устранение инцидентов в работе всех систем, включая системы аналитики и мониторинга технических показателей',
       'опыт работы в направлении devops не менее 3 лет, предпочтительно в проектах связанных с анализом данных',
       'опыт администрирования баз данных (mysql, postgresql и др',
       'опыт программирования на python и sql не менее 3 лет',
       'уверенные навыки администрирования операционных систем linux и 

In [88]:
import pandas as pd
from fuzzywuzzy import fuzz

# Создание пустого DataFrame для матрицы
matrix_df = pd.DataFrame(index=preprocessed_vacancy['requirements_predict'], columns=df_gb.index)

# Установка параметров для окна
threshold = 50  # Порог сходства

# Проходимся по каждому требованию и каждому элементу столбца 'other' и заполняем матрицу
for req in preprocessed_vacancy['requirements_predict']:
    for idx, other_text in df_gb['other'].items():
        max_similarity = 0
        window_size = len(req)
        for i in range(len(other_text) - window_size + 1):
            window = other_text[i:i+window_size].lower()
            similarity = fuzz.partial_ratio(req.lower(), window)
            max_similarity = max(max_similarity, similarity)
        if max_similarity >= threshold:
            matrix_df.at[req, idx] = True
        else:
            matrix_df.at[req, idx] = False

# Заполнение пропущенных значений False
matrix_df = matrix_df.fillna(False)


In [89]:
matrix_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
"администрирование и оптимизация инфраструктуры серверов, вкл",False,False,True,False,True,False,True,True,False,False,...,False,False,False,True,True,True,False,False,True,False
"bi-системы (например, fine bi) на linux и windows",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
управление базами данных для обеспечения их производительности и доступности,True,False,True,False,True,False,False,False,False,False,...,True,False,False,True,True,True,True,False,False,False
"разработка и поддержка систем мониторинга и логирования событий для всей инфраструктуры (сервера, базы данных, приложения)",False,False,False,False,True,True,False,True,False,False,...,False,False,False,True,True,True,False,False,True,False
"обеспечение безопасности систем, включая резервное копирование данных",True,True,True,True,True,True,True,True,False,False,...,False,False,True,False,False,True,True,True,True,True
"помощь разработчикам в настройке тестовых и разработческих сред, включая поддержку контейнеров и виртуализацию",False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
"диагностика и устранение инцидентов в работе всех систем, включая системы аналитики и мониторинга технических показателей",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"опыт работы в направлении devops не менее 3 лет, предпочтительно в проектах связанных с анализом данных",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"опыт администрирования баз данных (mysql, postgresql и др",False,True,True,True,True,True,True,True,False,False,...,False,False,True,True,True,False,True,True,True,True
опыт программирования на python и sql не менее 3 лет,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [71]:
other_text.lower()

'введение программирование расскажем спланировать обучение сохранить интерес получить максимум пользы всё успеть введение контроль версий практикум вы освоите ключевые операции системе создание клонирование репозитория слияние веток запрос истории изменений многое другое курс приблизит прохождению собеседования несколько вопросов обычно посвящены системе контроля версий знакомство языками программирования практикум познакомитесь языками программирования функциями массивами рекурсиями двумерными массивами узнаете нужно писать код знакомство базами данных познакомитесь понятием « базы данных » разберетесь видами основными подходами работе данными узнаете методы проектирования баз данных также способы модификации структуры итоги блока выбор специализации изучите колесо компетенций матрицу декарта познакомитесь специализациями выберете дальнейшее направление развития математика информатика программистов видеокурс дополнительный курс котором восполните необходимые базовые знания математике 

In [70]:
req

'понимание используемых технологий и способность аргументировать свои решения'

In [69]:
matrix_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
"основной стек: kotlin, mvp (moxy), mvvm, rxjava 3, coroutines, room, hilt + dagger, retrofit + okhttp, azure devops",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
разрабатывать новый функционал,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"уверенное знание kotlin, корутин и многопоточности",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"написание понятного, поддерживаемого и тестируемого кода",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
понимание используемых технологий и способность аргументировать свои решения,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [74]:
%pip install pyahocorasick

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 KB 858.7 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [77]:
end_index

NameError: name 'end_index' is not defined

In [75]:
import pandas as pd
from fuzzywuzzy import fuzz

# Создание пустого DataFrame для матрицы
matrix_df = pd.DataFrame(index=preprocessed_vacancy['requirements_predict'], columns=df_gb.index)

# Установка параметров для окна
window_size = 10  # Длина окна, которая должна быть примерно равной длине требования
threshold = 70  # Порог сходства

# Проходимся по каждому требованию и каждому элементу столбца 'other' и заполняем матрицу
for req in preprocessed_vacancy['requirements_predict']:
    for idx, other_text in df_gb['other'].items():
        max_similarity = 0
        for i in range(len(other_text) - window_size + 1):
            window = other_text[i:i+window_size].lower()
            similarity = fuzz.partial_ratio(req.lower(), window)
            max_similarity = max(max_similarity, similarity)
        if max_similarity >= threshold:
            matrix_df.at[req, idx] = True
        else:
            matrix_df.at[req, idx] = False

# Заполнение пропущенных значений False
matrix_df = matrix_df.fillna(False)

# Вывод результата
print(matrix_df)


                                                      0     1     2     3   \
основной стек: kotlin, mvp (moxy), mvvm, rxjava...  True  True  True  True   
разрабатывать новый функционал                      True  True  True  True   
уверенное знание kotlin, корутин и многопоточности  True  True  True  True   
написание понятного, поддерживаемого и тестируе...  True  True  True  True   
понимание используемых технологий и способность...  True  True  True  True   

                                                      4     5     6     7   \
основной стек: kotlin, mvp (moxy), mvvm, rxjava...  True  True  True  True   
разрабатывать новый функционал                      True  True  True  True   
уверенное знание kotlin, корутин и многопоточности  True  True  True  True   
написание понятного, поддерживаемого и тестируе...  True  True  True  True   
понимание используемых технологий и способность...  True  True  True  True   

                                                      8     9 

In [4]:
from joblib import dump
from binary import train_binary_trebovanyia_model, load_trained_clf

# Path to your CSV file
data_csv_path = "/home/andrew/works/VacancySkillsEstimator/notebooks/trebovanyia_full_data.csv"

# Training the model
train_binary_trebovanyia_model(data_csv_path)

binar_model = load_trained_clf('/home/andrew/works/VacancySkillsEstimator/notebooks/clf.joblib')

Accuracy: 0.8926829268292683
              precision    recall  f1-score   support

           0       0.85      0.91      0.88        88
           1       0.93      0.88      0.90       117

    accuracy                           0.89       205
   macro avg       0.89      0.89      0.89       205
weighted avg       0.89      0.89      0.89       205



[nltk_data] Downloading package punkt to /home/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
predictions

array([0, 0, 0])